<a href="https://colab.research.google.com/github/vanimalhotra22/vaniai/blob/main/hackathon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# 1. Load data
train = pd.read_csv("/content/Train_Data.csv")
test = pd.read_csv("/content/Test_Data.csv")

# 🔥 Drop rows where target is missing
train = train.dropna(subset=['age_group'])

# 2. Separate features and label
y = train['age_group']
X = train.drop(columns=['age_group'])

# 3. Combine for preprocessing
combined = pd.concat([X, test], axis=0)

# 4. Force numeric, coerce errors to NaN
combined = combined.apply(pd.to_numeric, errors='coerce')

# 5. Drop columns with all NaNs (optional safety step)
combined = combined.dropna(axis=1, how='all')

# 6. Impute missing values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
combined_imputed = pd.DataFrame(imputer.fit_transform(combined), columns=combined.columns)

# 7. Split again
X_clean = combined_imputed.iloc[:len(X)]
test_clean = combined_imputed.iloc[len(X):]

# 8. Train/test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_clean, y, test_size=0.2, random_state=42)

# 9. Model training
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 10. Validation
from sklearn.metrics import classification_report
val_preds = model.predict(X_val)
print(classification_report(y_val, val_preds))

# 11. Predict test
test_preds = model.predict(test_clean)

# 12. Submission
submission = pd.DataFrame({
    "id": test.index + 1,
    "age_group": test_preds
})
submission.to_csv("submission.csv", index=False)


              precision    recall  f1-score   support

       Adult       1.00      1.00      1.00       340
      Senior       1.00      1.00      1.00        51

    accuracy                           1.00       391
   macro avg       1.00      1.00      1.00       391
weighted avg       1.00      1.00      1.00       391



In [9]:
submission = pd.DataFrame({
    "id": test.index + 1,
    "age_group": test_preds
})
submission.to_csv("submission.csv", index=False)


In [10]:
from sklearn.metrics import classification_report

# Predict on validation set
val_preds = model.predict(X_val)

# Display classification report
print("Classification Report on Validation Set:\n")
print(classification_report(y_val, val_preds))


Classification Report on Validation Set:

              precision    recall  f1-score   support

       Adult       1.00      1.00      1.00       340
      Senior       1.00      1.00      1.00        51

    accuracy                           1.00       391
   macro avg       1.00      1.00      1.00       391
weighted avg       1.00      1.00      1.00       391



In [13]:
test_df = pd.read_csv("Test_Data.csv")  # Adjust if you're reading it with a different path
test_preds = model.predict(test_df)



In [14]:
# Fill missing values (like you did for train)
test = test.fillna(train.median(numeric_only=True))

# If categorical columns exist, encode them exactly like training


In [15]:
# Assuming you loaded test as `test` and handled preprocessing
test_preds = model.predict(test)

submission = pd.DataFrame({
    "id": test.index + 1,
    "age_group": test_preds
})

submission.to_csv("submission.csv", index=False)


In [17]:
# Ensure predictions are mapped correctly
test_preds = pd.Series(test_preds)  # in case it's a NumPy array
test_preds = test_preds.map({'Adult': 0, 'Senior': 1})

# Final submission
submission = pd.DataFrame({
    "id": test.index + 1,
    "age_group": test_preds.astype(int)
})
submission.to_csv("submission.csv", index=False)
